### OpenAI 파이썬 라이브러리 설치

In [ ]:
import os
from dotenv import load_dotenv

# API Key 환경변수 설정
load_dotenv()

### Chat Completions API를 이용한 함수 호출

1. 사용자 입력과 함수 정보를 Chat Completions API 모델로 보내기
2. 모델이 사용자의 입력에 따라 함수 호출을 위한 응답(JSON 객체)이나 일반 응답을 생성
3. 함수 호출을 위한 응답일 경우 JSON 객체의 구문을 분석하고, 인수가 있는 경우 인수를 사용해 함수를 호출
4. 함수 호출 결과를 기존 메시지에 추가해 Chat Completions API 모델로 다시 보내고 응답 결과를 사용자에게 표시

In [18]:
import openai
import json

# 호출 함수
def get_price_info(product_name):    
    product_price = {"키보드": "3만원", "마우스": "2만원", "모니터": "30만원"}
    
    price = product_price.get(product_name)  # 제품 이름을 입력해 가격 가져오기
    if price == None:  # 제품 가격이 없을 경우
        price = "해당 상품은 가격 정보가 없습니다."

    price_info = {
        "product_name": product_name,
        "price": price
    }

    return json.dumps(price_info) # JSON 형식으로 변환해 반환

# Chat Completions API에 사용자 입력과 함수 정보를 보내는 함수
def run_conversation_temp(user_query):   
    # 사용자 입력
    messages = [{"role": "user", "content": user_query}] 
        
    # 함수 정보 입력   
    functions = [                                        
        {
            "name": "get_price_info",
            "description": "제품 이름에 따른 가격 가져오기",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "제품 이름. 예를 들면, 키보드, 마우스",
                    },
                },
                "required": ["product_name"],
            },
        }
    ]

    # 1단계: 사용자 입력과 함수 정보를 Chat Completions API 모델로 보내기    
    response = openai.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = messages,
            functions = functions,
            function_call = "auto"
    )
    
    # 2단계: 응답 생성
    response_message = response.choices[0].message # 응답 메시지

    if response_message.function_call is not None: # 응답이 함수 호출인지 확인하기
        # 3단계: JSON 객체를 분석해 함수 이름과 인수를 추출한 후에 함수 호출
        # 주의: JSON 응답이 항상 유효하지 않을 수 있음
        
        # 함수 이름 추출
        function_name = response_message.function_call.name
        # 함수 호출을 위한 인수 추출
        function_args = json.loads(response_message.function_call.arguments)
        
        # 함수 호출 및 반환 결과 받기
        function_response = get_price_info(
            product_name=function_args.get("product_name") # 인수 지정
        )
        

        # 4단계: 함수 호출 결과를 기존 메시지에 추가하고, Chat Completions API 모델로 보내 응답받기

        # 함수 호출 결과를 기존 메시지에 추가하기
        messages.append(response_message)  # 기존 messages에 조력자 응답 추가
        messages.append(                   # 함수와 함수 호출 결과 추가
            {
                "role": "function",           # role: function으로 지정
                "name": function_name,        # name: 호출할 함수 이름 지정
                "content": function_response, # content: 함수 호출 결과 지정
            }
        )
        
        # 함수 호출 결과를 추가한 메시지를 Chat Completions API 모델로 보내 응답받기
        second_response = openai.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = messages,
        ) 
        return second_response.choices[0].message.content # 두 번째 응답 반환
    
    return response_message.content # 응답 메시지 반환

In [19]:
user_query = "마우스는 얼마인가요?" # 가격 정보 있음
response = run_conversation_temp(user_query)
json.dumps(response, ensure_ascii=False)

'"마우스의 가격은 20,000원 입니다."'

In [23]:
user_query = "HDD는 얼마인가요?" # 가격 정보 없음
response = run_conversation_temp(user_query)
json.dumps(response, ensure_ascii=False)

'"죄송합니다. HDD의 가격 정보를 찾을 수 없습니다. 해당 제품의 가격은 변동될 수 있으니 직접 온라인 쇼핑몰이나 전문 가전 제품 판매점을 방문하여 확인해주세요."'